<a href="https://colab.research.google.com/github/securemindorg/Educational-Examples/blob/main/Student_Loan_Payoff_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Student Loan Payoff Calculator

In [7]:
import math
import pandas as pd
from IPython.display import display

Specify loan starting amount, the annual percentage interest rate, and the normal current monthly payment.

In [11]:
current_total = 50000 # in dollars, don't include dollar signs or commas
annual_rate_percent = 6.5 # annual interest rate, don't include percent sign
regular_monthly_payment = 500 # total existing monthly payment, do not include commas or dollar signs

The following are functions for calculating the normal payment and scenarios

In [12]:
def calculate_regular_payoff(principal, annual_rate, monthly_payment):
    """
    Calculates the number of months (and years) to pay off a loan
    with daily compounding, using a fixed monthly payment.

    Parameters:
    - principal (float): The current total loan amount.
    - annual_rate (float): The annual interest rate (e.g., 0.05 for 5%).
    - monthly_payment (float): The regular fixed monthly payment.

    Returns:
    - tuple: (total_months, total_interest)
    """
    balance = principal
    total_months = 0
    total_interest = 0.0
    daily_rate = annual_rate / 365.0

    # Check if the payment is high enough to even cover the first month's interest
    # We use 30 days as a safe, average month length for the initial check.
    if monthly_payment <= balance * daily_rate * 30:
        print("--- WARNING: The monthly payment is too low to pay off the loan. ---")
        return None, None

    #

    while balance > 0:
        total_months += 1

        # Determine the number of days in the month (approximate for simplicity in a model)
        # Using a simple 30 days for every month in a general model for simplicity
        days_in_month = 30

        interest_accrued = 0.0

        # Daily Compounding Simulation for the month
        for day in range(days_in_month):
            interest_on_day = balance * daily_rate
            balance += interest_on_day
            interest_accrued += interest_on_day

        total_interest += interest_accrued

        # Apply the monthly payment
        principal_paid = monthly_payment - interest_accrued
        balance -= monthly_payment

        # Ensure the balance doesn't go negative on the final payment
        if balance < 0:
            total_interest += balance # Adjust for over-payment
            balance = 0
            break

    return total_months, total_interest

def simulate_payoff_scenario(principal, annual_rate, monthly_payment, extra_monthly=0, extra_yearly=0, one_time_extra=0):
    """
    Simulates early payoff with extra payments.

    Parameters:
    - principal, annual_rate, monthly_payment (as above)
    - extra_monthly (float): Extra amount paid each month.
    - extra_yearly (float): Extra lump sum paid once a year (e.g., tax refund).
    - one_time_extra (float): Extra lump sum paid at the start.

    Returns:
    - tuple: (total_months, total_interest)
    """
    balance = principal - one_time_extra # Apply one-time payment immediately
    total_months = 0
    total_interest = 0.0
    daily_rate = annual_rate / 365.0

    # Check for negative balance after one-time payment
    if balance <= 0:
        return 0, 0

    # Use the combined payment for the core loop
    total_monthly_payment = monthly_payment + extra_monthly

    while balance > 0:
        total_months += 1

        days_in_month = 30 # Simple approximation
        interest_accrued = 0.0

        # Daily Compounding Simulation for the month
        for day in range(days_in_month):
            interest_on_day = balance * daily_rate
            balance += interest_on_day
            interest_accrued += interest_on_day

        total_interest += interest_accrued

        # Apply the combined monthly payment
        payment_made = total_monthly_payment

        # Apply extra yearly payment on the 12th month of the year
        if total_months % 12 == 0:
            payment_made += extra_yearly

        balance -= payment_made

        # Final payment check
        if balance < 0:
            total_interest += balance # Adjust for over-payment
            balance = 0
            break

    return total_months, total_interest

When you run the next cell you'll be prompted for additional scenario input, for senario 4 you'll be prompted to for combined inputs.

In [13]:
print("STUDENT LOAN EARLY PAYOFF CALCULATOR")
print("------------------------------------")

annual_rate = annual_rate_percent / 100.0

# --- Regular Payoff Calculation ---

print("\n## Regular Payoff Calculation (Baseline)")
reg_months, reg_interest = calculate_regular_payoff(current_total, annual_rate, regular_monthly_payment)

if reg_months is not None:
    reg_years = reg_months / 12
    print(f"Regular Payoff Period: **{reg_months:,.0f} months** ({reg_years:.1f} years)")
    print(f"Total Interest Paid (Regular): **${reg_interest:,.2f}**")

# --- Scenario Planning ---

print("\n## Early Payoff Scenarios")

# Scenario 1: Extra Monthly Payment
extra_monthly = float(input("\nScenario 1: Extra amount to pay **monthly** ($): "))
s1_months, s1_interest = simulate_payoff_scenario(current_total, annual_rate, regular_monthly_payment, extra_monthly=extra_monthly)

# Scenario 2: Extra Yearly Payment
extra_yearly = float(input("Scenario 2: Extra amount to pay **yearly** ($): "))
s2_months, s2_interest = simulate_payoff_scenario(current_total, annual_rate, regular_monthly_payment, extra_yearly=extra_yearly)

# Scenario 3: One-Time Lump Sum Payment
one_time_extra = float(input("Scenario 3: One-time lump sum payment (today) ($): "))
s3_months, s3_interest = simulate_payoff_scenario(current_total, annual_rate, regular_monthly_payment, one_time_extra=one_time_extra)

# Scenario 4: All combined
extra_monthly_all = float(input("Scenario 4: Extra monthly (Combined) ($): "))
extra_yearly_all = float(input("Scenario 4: Extra yearly (Combined) ($): "))
one_time_extra_all = float(input("Scenario 4: One-time lump sum (Combined) ($): "))
s4_months, s4_interest = simulate_payoff_scenario(current_total, annual_rate, regular_monthly_payment,
                                                 extra_monthly=extra_monthly_all,
                                                 extra_yearly=extra_yearly_all,
                                                 one_time_extra=one_time_extra_all)


STUDENT LOAN EARLY PAYOFF CALCULATOR
------------------------------------

## Regular Payoff Calculation (Baseline)
Regular Payoff Period: **144 months** (12.0 years)
Total Interest Paid (Regular): **$21,593.23**

## Early Payoff Scenarios

Scenario 1: Extra amount to pay **monthly** ($): 500
Scenario 2: Extra amount to pay **yearly** ($): 5000
Scenario 3: One-time lump sum payment (today) ($): 10000
Scenario 4: Extra monthly (Combined) ($): 500
Scenario 4: Extra yearly (Combined) ($): 5000
Scenario 4: One-time lump sum (Combined) ($): 10000


Finally we will print the output table with all calculations

In [14]:
print("\n## Summary of Early Payoff Results (Formatted Table)")

data = []

# Function to calculate results and append to data list
def add_result(label, months, interest, extra_monthly=0, extra_yearly=0, one_time_extra=0):
    if months is not None:
        years = months / 12
        time_saved_years = reg_years - years
        interest_saved = reg_interest - interest
        total_extra_paid = (extra_monthly * 12 * years) + (extra_yearly * years) + one_time_extra

        data.append({
            'Scenario': label,
            'Payoff Time (Years)': years,
            'Time Saved (Years)': time_saved_years,
            'Total Interest Paid': interest,
            'Total Interest Saved': interest_saved,
        })

# Append all results
add_result("Regular (Baseline)", reg_months, reg_interest)
add_result(f"Extra Monthly (${extra_monthly:,.0f})", s1_months, s1_interest, extra_monthly=extra_monthly)
add_result(f"Extra Yearly (${extra_yearly:,.0f})", s2_months, s2_interest, extra_yearly=extra_yearly)
add_result(f"One-Time (${one_time_extra:,.0f})", s3_months, s3_interest, one_time_extra=one_time_extra)
add_result("Combined Scenario 4", s4_months, s4_interest,
            extra_monthly=extra_monthly_all,
            extra_yearly=extra_yearly_all,
            one_time_extra=one_time_extra_all)

# Create the DataFrame
df = pd.DataFrame(data)

# Format the columns for currency and decimals
df['Payoff Time (Years)'] = df['Payoff Time (Years)'].map('{:,.1f}'.format)
df['Time Saved (Years)'] = df['Time Saved (Years)'].map('{:,.1f}'.format)
df['Total Interest Paid'] = df['Total Interest Paid'].map('${:,.2f}'.format)
df['Total Interest Saved'] = df['Total Interest Saved'].map('${:,.2f}'.format)

# Display the HTML-formatted table
display(df)


## Summary of Early Payoff Results (Formatted Table)


,Scenario,Payoff Time (Years),Time Saved (Years),Total Interest Paid,Total Interest Saved
0,Regular (Baseline),12.0,0.0,"$21,593.23",$0.00
1,Extra Monthly ($500),4.9,7.1,"$7,707.15","$13,886.08"
2,"Extra Yearly ($5,000)",5.9,6.1,"$9,985.05","$11,608.18"
3,"One-Time ($10,000)",8.8,3.2,"$12,235.93","$9,357.30"
4,Combined Scenario 4,2.8,9.2,"$3,769.77","$17,823.46"
